# Augmentation Paraphreser

The idea behind this notebook is to augment the train dataset for the binary classifcation task of the repository. We will pick the **cleaned** version of the dataset, which is obtained through the cleaner notebook in the repository (spell checker not runned), and we will output a new dataset with double dimension. We will use a paraphreser (fine tuned pegasus) which will add for each tweet a paraphrased version. The model checkpoint can be found here: https://huggingface.co/tuner007/pegasus_paraphrase?text=Theres+an+emergency+evacuation+happening+now+in+the+building+across+the+street

In [5]:
!pip install transformers
!pip install sentencepiece
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### PARAPHRASER

In [2]:
# INITIALIZNG THE MODEL AND THE TOKENIZER FOR THE PARAPHRASIZER
import torch
import sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained("tuner007/pegasus_paraphrase")

model = AutoModelForSeq2SeqLM.from_pretrained("tuner007/pegasus_paraphrase").to(torch_device)

In [3]:
# FUNCTION TO GENERATE SYNONYM
def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

### COSINE SIMILARITY

In [8]:
# IMPORTING THE MODEL FOR SENTENCE SIMILARITY
import spacy
from spacy.cli import download

#a = download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

In [9]:
def sentence_similarity(sentence1, sentence2):
    doc1 = nlp(sentence1)
    doc2 = nlp(sentence2)
    
    # Compute cosine similarity between the document vectors
    cosine_sim = doc1.similarity(doc2)
    
    return cosine_sim


### FUNCTION FOR TEXT GENERATION

In [23]:
def paraphraser(original_sentence):
  
  # initiate a really high similarity score (1 is the maximum)
  best_score = 1

  # initiate a best sentence
  best_sentence = original_sentence

  # get list of three sentences
  list_of_sentences = get_response(original_sentence,10,10)

  # get the sentence with the least similar sentence

  for sentence in list_of_sentences:
    
    # compute the similarity score
    sim_score = sentence_similarity(original_sentence,sentence)

    # check if less similar
    if sim_score < best_score:

      # update best sentence
      best_sentence = sentence
  
  return best_sentence


In [26]:
paraphraser(text)

<ipython-input-9-a54e125b9293>:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  cosine_sim = doc1.similarity(doc2)


'May Allah help us all with the reason of this earthquake.'

### APPLYING TO INPUT DATASET

In [32]:
import pandas as pd
# Load the dataset into a pandas DataFrame

df = pd.read_csv("cleaned_train_df.csv")
df = df.head(10)

# Create a new dataframe with exact duplicates of each row, but with the text column replaced by the paraphrased version
df_new = pd.DataFrame(columns=df.columns)
for index, row in df.iterrows():
    new_text = paraphraser(row["text"])
    new_row = row.copy()
    new_row["text"] = new_text
    df_new = df_new.append(new_row)

# Combine the original and new dataframes
df_combined = pd.concat([df, df_new], axis=0)

<ipython-input-9-a54e125b9293>:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  cosine_sim = doc1.similarity(doc2)
<ipython-input-32-dc2cc03fb5c3>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_new = df_new.append(new_row)
<ipython-input-9-a54e125b9293>:6: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're

In [ ]:
df_combined.to_csv("augmented_cleaned_train_df.csv")